# Webscrapping BoardGameGeek.com
## Project scope
In this project we set out to get a list of top 10 000 board games on BoardGameGeek (with links), and then use this list to scrap individual pages for more detailed info.  
## Execution
Scrapping the top list was fast because we were able to work using requests.get(). Scrapping the individual board games has proven more difficult. The website uses javascript to dynamically load the important part of the page, so using the simple requests.get() wasn't a success.   
To overcome this challenge, we needed to use a emulated browser to execute the JS code. We chose Selenium, because we had some experience with it during the previous semester. Since this process is much slower, we decided to scrap only 100 individual pages. We also didn't want to burden the server, but since BGG is a very popular website, the additional 1 000 visits probably wouldn’t make a difference anyway. 
## Summary


# Scrapping the summary page

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re

In [16]:
headers = {'User-Agent': 'Animated Winter'}

#Create an empty dataframe to hold the results
summaryTable = pd.DataFrame()

# Each page has 100 boardgames, so we want to loop 100 times
for i in range(1,101):
    # Get the webpage
    r = requests.get('https://boardgamegeek.com/browse/boardgame/page/{}?sort=rank'.format(i), headers=headers)
    
    # Convert to BS
    bs = BeautifulSoup(r.text)
    
    # Find the right table
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('cellpadding'))
    tableHTML = table[0]
    
    # Extract all the links
    urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]
    urls = pd.DataFrame(urls)
    
    # Convert the HTML of the table to pandas (very usefull function)
    pandasTable = pd.read_html(str(tableHTML), header=0)[0]
    
    # Format the links
    urls[0] = urls[0].astype(str)
    urls = urls[urls[0].str.contains("/boardgame/")][1:]
    urls = urls[~urls[0].str.contains("/marketplace/")]
    
    # Remove any duplicates
    urls = pd.unique(urls[0])
    urls = pd.DataFrame(urls)
    
    # Merge the links into the DF
    pandasTable['link'] = urls
    
    # Drop unused columns
    pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)
    
    # Create or extend the existing summary table
    if summaryTable.empty:
        summaryTable = pandasTable
    else:
        summaryTable = pd.concat([summaryTable, pandasTable],ignore_index=True)

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link
0,1,Gloomhaven (2017),8.608,8.90,24863,/boardgame/174430/gloomhaven
1,2,Pandemic Legacy: Season 1 (2015),8.488,8.64,31084,/boardgame/161936/pandemic-legacy-season-1
2,3,Through the Ages: A New Story of Civilization ...,8.258,8.53,15653,/boardgame/182028/through-ages-new-story-civil...
3,4,Terraforming Mars (2016),8.243,8.40,38741,/boardgame/167791/terraforming-mars
4,5,Twilight Struggle (2005),8.173,8.32,34385,/boardgame/12333/twilight-struggle
5,6,Star Wars: Rebellion (2016),8.164,8.45,16567,/boardgame/187645/star-wars-rebellion
6,7,Gaia Project (2017),8.121,8.54,9239,/boardgame/220308/gaia-project
7,8,Scythe (2016),8.120,8.29,39504,/boardgame/169786/scythe
8,9,Twilight Imperium (Fourth Edition) (2017),8.085,8.73,6670,/boardgame/233078/twilight-imperium-fourth-edi...
9,10,Great Western Trail (2016),8.081,8.29,19186,/boardgame/193738/great-western-trail


In [18]:
# We have a summary of 10 000 most popular games on BoardGameGeek, with some basic stats. 
# To get the full link add https://boardgamegeek.com before the string.
display(summaryTable)

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link
0,1,Gloomhaven (2017),8.608,8.90,24863,/boardgame/174430/gloomhaven
1,2,Pandemic Legacy: Season 1 (2015),8.488,8.64,31084,/boardgame/161936/pandemic-legacy-season-1
2,3,Through the Ages: A New Story of Civilization ...,8.258,8.53,15653,/boardgame/182028/through-ages-new-story-civil...
3,4,Terraforming Mars (2016),8.243,8.40,38741,/boardgame/167791/terraforming-mars
4,5,Twilight Struggle (2005),8.173,8.32,34385,/boardgame/12333/twilight-struggle
5,6,Star Wars: Rebellion (2016),8.164,8.45,16567,/boardgame/187645/star-wars-rebellion
6,7,Gaia Project (2017),8.121,8.54,9239,/boardgame/220308/gaia-project
7,8,Scythe (2016),8.120,8.29,39504,/boardgame/169786/scythe
8,9,Twilight Imperium (Fourth Edition) (2017),8.085,8.73,6670,/boardgame/233078/twilight-imperium-fourth-edi...
9,10,Great Western Trail (2016),8.081,8.29,19186,/boardgame/193738/great-western-trail


In [25]:
# For Python
summaryTable.to_pickle("Summary.p")

# For Excel
summaryTable.to_csv("Summary.csv", sep=';', decimal = ',')

# Scrapping the individual pages

In [7]:
driver = webdriver.Firefox()

In [11]:
detailedTable = pd.DataFrame(columns=['Title','Average Rating','Number of ratings','Std.dev','Complexity','Number of Comments','Fans','Page Views','Overall Ranking','All Time Plays','Played This Month', 'Own', 'Previously Owned','For Trade','Want in Trade','Wishlist','Has Parts','Want Parts'])
detailedTable2 = pd.DataFrame()

for j in range(100):
    driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][j]))
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource, "lxml")
    data = bs.findAll('div',{'class':'outline-item-description'})
    
    data2 = []
    for i in data:
        data2.append(i.get_text().strip())

    for i in range(len(data2)):
        data2[i] = data2[i].replace(",","")
        #data2[i] = data2[i].replace(".",",")

    data2[3] = data2[3][0:4]
    data2[0] = float(data2[0])
    data2[1] = int(data2[1]) 
    data2[2] = float(data2[2])
    data2[3] = float(data2[3]) 

    for i in range(len(data2[4:])+1):
        i = i+3
        data2[i] = int(data2[i])

    if len(data2) == 19:
        data2.pop(8)
        data2.pop(8)
    else:
        data2.pop(8)
    
    data2.insert(0,summaryTable['Title'][j]) 
    detailedTable.loc[len(detailedTable)] = data2

In [12]:
display(detailedTable)

,Title,Average Rating,Number of ratings,Std.dev,Complexity,Number of Comments,Fans,Page Views,Overall Ranking,All Time Plays,Played This Month,Own,Previously Owned,For Trade,Want in Trade,Wishlist,Has Parts,Want Parts
0,Gloomhaven (2017),8.901,24863,1.60,3,4937,4769,8121209,1,177562,2466,40792,1264,216,1344,12154,18,7
1,Pandemic Legacy: Season 1 (2015),8.642,31084,1.61,2,4959,2247,2822351,2,178979,664,49456,3194,238,780,8483,10,5
2,Through the Ages: A New Story of Civilization ...,8.533,15653,1.49,4,2282,1113,2235683,3,43742,386,19372,1085,250,1037,6436,6,4
3,Terraforming Mars (2016),8.403,38741,1.36,3,5856,3092,4120292,4,187454,3045,49782,1419,228,2068,12969,15,6
4,Twilight Struggle (2005),8.324,34385,1.59,3,7738,3726,4339022,5,89047,334,47302,3204,780,1342,8913,36,28
5,Star Wars: Rebellion (2016),8.447,16567,1.38,3,2681,1670,2746485,6,24705,170,25232,912,188,1350,7945,4,6
6,Gaia Project (2017),8.541,9239,1.43,4,1389,821,1345459,7,27821,461,11380,347,54,1214,6172,1,2
7,Scythe (2016),8.287,39504,1.43,3,6217,4264,5017886,8,124795,1295,51381,1703,300,1880,13293,27,10
8,Twilight Imperium (Fourth Edition) (2017),8.726,6670,1.57,4,1106,940,1310082,9,8916,162,9329,175,43,728,4857,0,1
9,Great Western Trail (2016),8.287,19186,1.25,3,2606,955,1645368,10,58428,714,23465,894,114,1626,6988,6,3


In [346]:
detailedTable.to_pickle("Detailed.p")

# Data analysis